In [7]:
import h5py
import numpy as np

with h5py.File('/home/liw324/code/Segment/EarthVQA/log/sfpnr50/test_features/0.hdf5', 'r') as f:
    print(f.keys())
    # print(f['feature'].shape)
    # print(f['feature'][0])
    # print(f['feature'][0].shape)
    print(f['pred_mask'].shape)
    print(f['pred_mask'][0])
    

<KeysViewHDF5 ['feature', 'pred_mask']>
(1024, 1024)
[2 2 2 ... 7 7 7]


In [4]:
import numpy as np
import PIL.Image as Image

pic = Image.open("/home/liw324/code/Segment/EarthVQA/dataset/Train/masks_png/0.png")
pic_numpy = np.array(pic)
print(pic_numpy)

[[2 2 2 ... 1 1 1]
 [2 2 2 ... 1 1 1]
 [2 2 2 ... 1 1 1]
 ...
 [3 3 3 ... 7 7 7]
 [3 3 3 ... 7 7 7]
 [3 3 3 ... 7 7 7]]
